In [ ]:
import os
import subprocess
import csv
from datetime import datetime

# === Konfiguration ===
REPO_PATH = "/home/pi/Robo_challenge"  # Pfad zum lokalen Git-Repository
SUBMISSION_DIR = os.path.join(REPO_PATH, "submissions")
TEAM_NAME = "Zumi123"  # <-- Teamnamen hier anpassen

# === Hilfsfunktionen ===

def get_unique_filename():
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    return "{}_result_{}.csv".format(TEAM_NAME, timestamp)

def create_submission_file():
    """Erzeugt eine neue CSV-Datei mit Kopfzeile für Logs."""
    if not os.path.exists(SUBMISSION_DIR):
        os.makedirs(SUBMISSION_DIR)

    file_name = get_unique_filename()
    file_path = os.path.join(SUBMISSION_DIR, file_name)

    with open(file_path, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Zeit", "Aktion"])

    print("📄 Neue Logdatei erstellt:", file_path)
    return file_path

def log_aktion(filepath, aktion="Aktion nicht angegeben"):
    """Fügt eine neue Zeile mit Zeitstempel und Aktion hinzu."""
    now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(filepath, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([now, aktion])
    print("📝 Aktion geloggt:", aktion)

def upload_submission(file_path):
    """Fügt Datei zu Git hinzu, commitet und pusht sie."""
    try:
        COMMIT_MESSAGE = "Submission by {} - {}".format(TEAM_NAME, datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        subprocess.run(["git", "-C", REPO_PATH, "add", file_path], check=True)
        subprocess.run(["git", "-C", REPO_PATH, "commit", "-m", COMMIT_MESSAGE], check=True)
        subprocess.run(["git", "-C", REPO_PATH, "push", "origin", "main"], check=True)
        print("✅ Datei erfolgreich gepusht:", os.path.basename(file_path))
    except subprocess.CalledProcessError as e:
        print("❌ Git Error:", e)
    except Exception as e:
        print("❌ Allgemeiner Fehler:", e)

def upload_face_image(image_path):
    """Gespeichertes Gesicht als PNG in Git hochladen."""
    try:
        COMMIT_MESSAGE = "Gesicht erkannt von {} - {}".format(TEAM_NAME, datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
        subprocess.run(["git", "-C", REPO_PATH, "add", image_path], check=True)
        subprocess.run(["git", "-C", REPO_PATH, "commit", "-m", COMMIT_MESSAGE], check=True)
        subprocess.run(["git", "-C", REPO_PATH, "push", "origin", "main"], check=True)
        print("✅ Gesicht erfolgreich gepusht:", os.path.basename(image_path))
    except subprocess.CalledProcessError as e:
        print("❌ Git Error beim Bild-Upload:", e)
    except Exception as e:
        print("❌ Allgemeiner Fehler beim Bild-Upload:", e)



In [ ]:
from zumi.zumi import Zumi
import time

from zumi.util.screen import Screen

from zumi.util.camera import Camera
from zumi.util.vision import Vision

from zumi.personality import Personality

from zumi.protocol import Note

import cv2


zumi = Zumi()
zumi.reset_gyro()
camera = Camera()
screen = Screen()
personality = Personality(zumi, screen)
vision = Vision()

screen.happy()

#GLOBAL VARs
whiteTreshold = 150
circRounds = 1
speed = 15
jump_back_threshold = 20

logfile = create_submission_file()
face_count = 0
object_count = 0
qr_count = 0
ziel_ankunft = 0
image_path = 0

#=====NEU=========
def end_course():
    global start_time
    global image_path
    zumi.stop()
    end_time = time.time()
    
    log_aktion(logfile, aktion="Ende der Fahrt")
    
    komp_time = int(end_time - start_time)
    minuten, sekunden = divmod(komp_time, 60)
    zeit_string = "{:02d}:{:02d}".format(minuten, sekunden)
    
    log_aktion(logfile, aktion="Total time = " + zeit_string)
    screen.draw_text_center("Parcours abgeschlossen.")
    print("Parcours abgeschlossen. Gesamtzeit:", zeit_string)
    camera.close()
    upload_submission(logfile)  # Upload the latest submission
    # Gesicht ins Git-Repo pushen
    upload_face_image(image_path)

    while True:
        zumi.stop()
        time.sleep(1)

def corr_line(heading):
    head = heading
    ir_readings = zumi.get_all_IR_data()
    
    # === NEU: Frontsensor-Prüfung auf Hindernisse ===
    left_ir_front = ir_readings[5]
    right_ir_front = ir_readings[0]
    front_threshold = 50  # Schwellenwert für Hindernis (anpassbar)

    if left_ir_front < front_threshold or right_ir_front < front_threshold:
        print("Hindernis im corr_line erkannt – Objekt voraus!")
        zumi.stop()
        object_ahead(zumi, screen, personality)
        return head  # oder ggf. neuen Startwert wie 0

    # === Bestehende Linienkorrektur mit unteren Sensoren ===
    left_ir_bottom = ir_readings[3]
    right_ir_bottom = ir_readings[1]

    if left_ir_bottom > whiteTreshold and right_ir_bottom <= whiteTreshold:
        head += 5
    elif right_ir_bottom > whiteTreshold and left_ir_bottom <= whiteTreshold:
        head -= 5
    elif left_ir_bottom <= whiteTreshold and right_ir_bottom <= whiteTreshold:
        zumi.stop()
        head = "weiss"

    return head
    
def smooth(raw_values):
    smoothed = []
    for i in range(len(raw_values)):
        if i < 2:  # Handle first two elements
            window = raw_values[0:i+1]
        else:
            window = raw_values[i-2:i+1]
        avg = sum(window) / len(window)
        smoothed.append(avg)
    return smoothed

def search_curve(zumi):
    zumi.stop()
    print("Linie verloren – starte gezielte Suche...")

    zumi.reset_gyro()
    time.sleep(0.1)

    # === Suche nach Westen/links (90 Grad) ===
    zumi.signal_left_on()
    zumi.turn(90)
    zumi.signal_left_off()
    time.sleep(0.3)
    ir_readings = zumi.get_all_IR_data()
    right_ir_bottom = ir_readings[1]
    left_ir_bottom = ir_readings[3]
    

    if left_ir_bottom > whiteTreshold and right_ir_bottom > whiteTreshold:
        print("Linie im Westen gefunden.")
        zumi.reset_gyro()
        log_aktion(logfile, aktion="Linkskurve Zeile 95")
        return 0
    if left_ir_bottom > whiteTreshold:
        # Versuch Feinjustierung nach links
        print("Linker Sensor sieht Linie – feinjustiere weiter nach links...")
        zumi.turn_left(20)
        time.sleep(0.3)
        ir_readings = zumi.get_all_IR_data()
        if ir_readings[1] > whiteTreshold and ir_readings[3] > whiteTreshold:
            print("Linie nach Feinjustierung im westen gefunden.")
            zumi.reset_gyro()
            return 0

    # === Suche nach Osten/rechts (-90 Grad) ===
    zumi.signal_right_on()
    zumi.turn(-90)  # -90 zurück + 20 zusätzliche Korrekturrichtung
    time.sleep(0.3)
    zumi.signal_right_off()
    ir_readings = zumi.get_all_IR_data()
    right_ir_bottom = ir_readings[1]
    left_ir_bottom = ir_readings[3]

    
    if left_ir_bottom > whiteTreshold and right_ir_bottom > whiteTreshold:
        print("Linie im Osten gefunden.")
        zumi.reset_gyro()
        log_aktion(logfile, aktion="Rechts kurve Zeile 121")
        return 0
    if right_ir_bottom > whiteTreshold:
        # Feinjustierung weiter nach rechts
        print("Rechter Sensor sieht Linie – feinjustiere weiter nach rechts...")
        zumi.turn_right(20)
        time.sleep(0.3)
        ir_readings = zumi.get_all_IR_data()
        if ir_readings[1] > whiteTreshold and ir_readings[3] > whiteTreshold:
            print("Linie nach Feinjustierung im Osten gefunden.")
            zumi.reset_gyro()
            return 0

    # === Keine Linie gefunden ===
    print("Keine Linie erkannt.")
    return None

def follow_circle(zumi, direction="right"):
    global object_count
    global qr_count
    global face_count
    camera.close()
    heading = 0
    turn_count = 0
    object_count_ = object_count - qr_count - qr_count
    circRounds = object_count_ - face_count
    max_turns = 4 * circRounds
    screen.draw_text_center("Kreisverkehrfahren")
    time.sleep(0.5)
    screen.draw_text_center(str(circRounds))
    rev = "right"
    
    while True and circRounds>=1: #<---------Neu
        log_aktion(logfile, aktion="Starte Kreisverkehr Zeile 152")
        # === 3. Linienfolge wie gehabt ===
        heading = corr_line(heading)
        if heading == "weiss":
            print("Linie verloren – Turn QR Circle richtung...")

            if direction == "right":
                zumi.signal_right_on()
                zumi.turn(-90)  # -90 zurück
                zumi.signal_right_off()
                time.sleep(0.3)
                zumi.reset_gyro()
                new_heading = 0
                rev = "left"

            elif direction == "left":
                zumi.signal_left_on()
                zumi.turn(90)  # -90 zurück
                zumi.signal_left_off()
                time.sleep(0.3)
                zumi.reset_gyro()
                new_heading = 0
                rev = "right"

            turn_count += 1
            print("Abbiegevorgang:", turn_count, "von", max_turns)
            screen.draw_text_center("Abbiegevorgang: {} von {}".format(turn_count, max_turns))

            if new_heading is not None:
                heading = new_heading
                if turn_count >= max_turns:
                    while True:
                        print("LOOP AFTER CIRCLE")
                        heading = corr_line(heading)
                        if heading == "weiss":
                            print("Weiße Fläche erkannt")
                            log_aktion(logfile, aktion = "Kreisverkehr verlassen Zeile 189")
                            if rev == "left":
                                zumi.signal_left_on()
                                zumi.turn(90)
                                zumi.signal_left_off()
                                zumi.reset_gyro()
                                
                            elif rev == "right":
                                zumi.signal_right_on()
                                zumi.turn(-90)
                                zumi.signal_right_off()
                                zumi.reset_gyro()
                                
                            return 0
                        
                        zumi.go_straight(10, heading)  # langsam geradeaus
                        time.sleep(0.005)
                continue
            else:
                print("Suche gescheitert – Programmabbruch.")
                break
        
        print("HEADING CIRCLE FOLLOW END: ", heading)
        zumi.go_straight(speed, heading)
        time.sleep(0.02)
    if circRounds <= 1:
        log_aktion(logfile, "Ich darf keinen kreisverkehr fahren, da ich gleichviele Objekte und Gesichter erkannte")
    zumi.stop()
    

def follow_line(zumi, abs_ir_threshold=50):
    heading = 0
    start_time = time.time()
    print("HEADING START: ", heading)

    # Historie mit Startwerten füllen
    ir_readings = zumi.get_all_IR_data()
    right_ir_history = [ir_readings[0]] * 10
    left_ir_history = [ir_readings[5]] * 10

    while True:
        ir_readings = zumi.get_all_IR_data()
        left_ir_front = ir_readings[5]
        right_ir_front = ir_readings[0]

        # === 1. Sofortiger IR-Stopp bei direkter Objekterkennung ===
        #if left_ir_front < abs_ir_threshold or right_ir_front < abs_ir_threshold:
        #    zumi.stop()
        #    print("SOFORT-STOPP: Objekt sehr nah (IR-Werte: L=", (left_ir_front), "R=",(right_ir_front))
        #    object_ahead(zumi, screen, personality)
        #    continue

        # Historie aktualisieren
        right_ir_history.append(right_ir_front)
        left_ir_history.append(left_ir_front)
        if len(right_ir_history) > 10:
            print(right_ir_history)
            right_ir_history.pop(0)
        if len(left_ir_history) > 10:
            print(left_ir_history)
            left_ir_history.pop(0)

        # NEW: Apply noise filtering
        right_smoothed = smooth(right_ir_history)
        left_smoothed = smooth(left_ir_history)

        # Kumulierte Deltas berechnen (using smoothed values)
        delta_right_total = sum([
            abs(right_smoothed[i+1] - right_smoothed[i])
            for i in range(len(right_smoothed)-1)
        ])
        delta_left_total = sum([
            abs(left_smoothed[i+1] - left_smoothed[i])
            for i in range(len(left_smoothed)-1)
        ])

        # === 3. Linienfolge wie gehabt ===
        heading = corr_line(heading)
        if heading == "weiss":
            print("Linie verloren – suche neue Richtung...")
            new_heading = search_curve(zumi)
            if new_heading is not None:
                heading = new_heading
                continue
            else:
                print("Suche gescheitert – Programmabbruch.")
                break
                
        # === 2. Reagiere auf kumulativen Sprung ===
        if delta_left_total > jump_back_threshold or delta_right_total > jump_back_threshold:
            zumi.stop()
            print("KUMULATIVER IR-Sprung erkannt! ΔL=" + str(delta_left_total) + " ΔR=" + str(delta_right_total))
            object_ahead(zumi, screen, personality)
            # Historie mit Startwerten füllen
            print("HEADING KUM: ", heading)
            
            ### GEAENDERT
            ir_readings = zumi.get_all_IR_data()
            right_ir_history.clear()
            left_ir_history.clear()
            right_ir_history = [ir_readings[0]] * 10
            left_ir_history = [ir_readings[5]] * 10
            zumi.reset_gyro()
            heading = 0
            print("HEADING NEW KUM: ", heading)
            continue
        
        print("HEADING FOLLOW END: ", heading)
        zumi.go_straight(speed, heading)
        time.sleep(0.02)

    zumi.stop()

def read_qr_message(image):
    qr_code = vision.find_QR_code(image)
    if qr_code is not None:
        message = vision.get_QR_message(qr_code)
        screen.draw_text_center("QR-Codeerkannt")
        print("QR-Code erkannt:", message)
        return message
    else:
        return None
    
def handle_qr_action(message, zumi, screen, personality):
    emotions = ["Zumi is happy today!", "Zumi is angry today!", "Zumi is celebrating today!"]
    directions = ["Turn Right", "Turn Left", "Right Circle", "Left Circle"]
    
    spin = ["3 x 360 left, emotion: happy", "1 x 360 right, emotion: happy", "2 x 360 left, emotion: celebrating", "2 x 360 right, emotion: celebrating", "1 x 360 left, emotion: angry", "2 x 360 left, emotion: angry"]
    
    log_aktion(logfile, "Zeile 315 Qr-Code"+ str(message))
    global qr_count
    qr_count+=1
    global ziel_ankunft #<-----------------Neu

    if message in directions:
        print("Fahre langsam vor zur Kreuzung...")
        zumi.reset_gyro()
        while True:
            head = corr_line(0)
            if head == "weiss":
                print("Weiße Fläche erkannt – führe QR-Aktion aus")
                break

            zumi.go_straight(10, head)  # langsam geradeaus
            time.sleep(0.005)

        if message == "Turn Right":
            print("QR-Aktion: Rechts abbiegen")
            log_aktion(logfile, "Zeile 334 nach rechts abbiegen")
            zumi.signal_right_on()
            zumi.turn_right(-90)
            zumi.signal_right_off()

        elif message == "Turn Left":
            print("QR-Aktion: Links abbiegen")
            log_aktion(logfile, "Zeile 341 nach links abbiegen")
            zumi.signal_left_on()
            zumi.turn_left(90)
            zumi.signal_left_off()


        elif message == "Right Circle":
            print("QR-Aktion: Rechtskreis")
            log_aktion(logfile, "Zeile 349 rechter Kreis")
            follow_circle(zumi, direction="right")

        elif message == "Left Circle":
            print("QR-Aktion: Linkskreis")
            log_aktion(logfile, "Zeile 354 linker Kreis")
            follow_circle(zumi, direction="left")


    elif message in emotions:
        print("Fahre langsam zur Zielzone...")
        zumi.reset_gyro()
        while True:
            head = corr_line(0)
            if head == "weiss":
                print("Weiße Fläche erkannt – führe QR-Aktion aus")
                break

            zumi.go_straight(10, head)  # langsam geradeaus
            time.sleep(0.005)

        while True:
            head = corr_line(0)
            if head == "weiss":
                print("Weiße Zielfläche erreicht – führe Emotion aus")
                break

            zumi.go_straight(10, head)
            time.sleep(0.05)

        if message == "Zumi is happy today!":
            print("QR-Aktion: Freude anzeigen")
            log_aktion(logfile, "Zeile 381 Freude Zeigen")
            screen.happy()
            personality.happy()

        elif message == "Zumi is angry today!":
            print("QR-Aktion: Wut anzeigen")
            log_aktion(logfile, "Zeile 387 Wütend")
            screen.angry()
            personality.angry()

        elif message == "Zumi is celebrating today!":
            print("QR-Aktion: Feier-Modus")
            log_aktion(logfile, "Zeile 393 Feiern")
            personality.celebrate()

        print("Zumi ist am Ziel angekommen und bleibt stehen.")
        log_aktion(logfile, "Zeile 397 Ziel ankunft")
        ende = end_course()
        #while True:
        #    zumi.stop()
        #    time.sleep(1)  # Endlosschleife = vollständiger Stopp


    # wurde entfernt aus den missions
    #elif message == "Park":
    #    None
    # wurde entfernt aus den missions
    #elif message == "Drive Round Road":
    #    None
    elif message in spin:
        camera.close()
        print("Fahre langsam zur Zielzone...")
        
        zumi.forward(speed=2, duration=2)
        
        if message == "3 x 360 left, emotion: happy":
            print("QR-Aktion: 3 x 360 left, emotion: happy")
            for _ in range(3):
                zumi.signal_left_on()
                zumi.turn_left(360, 3)
            zumi.signal_left_off()
            time.sleep(2)
            print("QR-Aktion: happy")
            screen.happy()
            zumi.forward(2)
            log_aktion(logfile, aktion="Zeile 431 3 x 360 left, emotion: happy")
            
        elif message == "1 x 360 right, emotion: happy":
            print("QR-Aktion: 1 x 360 right, emotion: happy")
            for _ in range(1):
                zumi.signal_right_on()
                zumi.turn_right(360, 3)
            zumi.signal_right_off()
            time.sleep(2)
            print("QR-Aktion: happy")
            screen.happy()
            zumi.forward(2)
            log_aktion(logfile, aktion="Zeile 442 1 x 360 right, emotion: happy")
        
        elif message == "2 x 360 left, emotion: celebrating":
            print("QR-Aktion: 2 x 360 left, emotion: celebrating")
            for _ in range(2):
                zumi.signal_left_on()
                zumi.turn_left(360, 3)
            zumi.signal_left_off()
            time.sleep(2)
            print("QR-Aktion: celebrating")
            personality.celebrate()
            zumi.forward(2)
            log_aktion(logfile, aktion="Zeile 453 2 x 360 left, emotion: celebrating")
        
        elif message == "2 x 360 right, emotion: celebrating":
            print("QR-Aktion: 2 x 360 right, emotion: celebrating")
            for _ in range(2):
                zumi.signal_right_on()
                zumi.turn_right(360, 3)
            zumi.signal_right_off()
            time.sleep(2)
            print("QR-Aktion: celebrating")
            personality.celebrate()
            zumi.forward(2)
            log_aktion(logfile, aktion="Zeile 464 2 x 360 right, emotion: celebrating")
            
        elif message == "1 x 360 left, emotion: angry":
            print("QR-Aktion: 1 x 360 left, emotion: angry")
            zumi.signal_left_on()
            zumi.turn_left(360, 3)
            zumi.signal_left_off()
            time.sleep(2)
            print("QR-Aktion: angry")
            screen.angry()
            zumi.forward(2)
            log_aktion(logfile, aktion="Zeile 474 1 x 360 left, emotion: angry")
            
        elif message == "2 x 360 left, emotion: angry":
            print("QR-Aktion: 2 x 360 left, emotion: angry")
            for _ in range(2):
                zumi.signal_left_on()
                zumi.turn_left(360, 3)
            zumi.signal_left_off()
            time.sleep(2)
            print("QR-Aktion: angry")
            screen.angry()
            zumi.forward(2)
            log_aktion(logfile, aktion="Zeile 485 2 x 360 left, emotion: angry")
    
    elif message == "Stop":
        print("QR-Aktion: Stop – Zumi hält an")
        log_aktion(logfile, "Zeile 482 Stopp")
        zumi.stop()
        time.sleep(1.5) # hier einstellen wie lange gestoppt wird
        return
    
    else:
        print("QR-Inhalt unbekannt:", message)
        log_aktion(logfile, aktion="Zeile 489 QR-Code unbekannt")
        
    time.sleep(0.5)  # kleine Pause nach der Aktion

def object_ahead(zumi, screen, personality):
    global object_count
    object_count += 1
        
    zumi.stop()
    time.sleep(0.7)

    ir_readings = zumi.get_all_IR_data()
    base_left_ir = ir_readings[5]
    base_right_ir = ir_readings[0]

    print("Hindernis erkannt – Stop & Foto")
    screen.draw_text_center("Hindernis erkannt – Stop & Foto")
    zumi.play_note(Note.C4, 500)
    zumi.play_note(Note.C5, 500)  # Spielt mittleres C für 500ms
    zumi.brake_lights_on()
    screen.angry()
    personality.angry()
    time.sleep(0.7)
    zumi.brake_lights_off()
    log_aktion(logfile, str(object_count) + ". Hindernis erkannt Zeile 513")
    print("IR vorne: IR0 =", base_right_ir, "IR5 =", base_left_ir)

    try:
        camera.start_camera()
        time.sleep(0.9)
        image = camera.capture()
        camera.show_image(image)
        message = read_qr_message(image)
        time.sleep(1.0)

        if message is not None:
            print("QR erkannt – führe Aktion aus")
            screen.draw_text_center("QR erkannt – führe Aktion aus")
            handle_qr_action(message, zumi, screen, personality)
            time.sleep(1.0)
        else:
            print("Kein QR-Code – warte auf Objekt-Entfernung...")
            screen.draw_text_center("Kein QR-Code – warte auf Objekt-Entfernung...")

            # Historie für Differenzsprung rückwärts (FIFO-Puffer)
            left_ir_history = [base_left_ir] * 10
            right_ir_history = [base_right_ir] * 10

            time.sleep(0.5)

            while True:
                ir_readings = zumi.get_all_IR_data()
                left_ir = ir_readings[5]
                right_ir = ir_readings[0]

                left_ir_history.append(left_ir)
                right_ir_history.append(right_ir)
                if len(left_ir_history) > 10:
                    left_ir_history.pop(0)
                if len(right_ir_history) > 10:
                    right_ir_history.pop(0)

                # Kumulierte Differenzen berechnen (über Zwischenwerte)
                delta_left_total = sum([
                    left_ir_history[i+1] - left_ir_history[i]
                    for i in range(len(left_ir_history)-1)
                ])
                delta_right_total = sum([
                    right_ir_history[i+1] - right_ir_history[i]
                    for i in range(len(right_ir_history)-1)
                ])

                print("IR: L=" + str(left_ir) + ", R=" + str(right_ir) +
                      " | ΔL=" + str(delta_left_total) + ", ΔR=" + str(delta_right_total))

                if abs(delta_left_total) > jump_back_threshold or abs(delta_right_total) > jump_back_threshold:
                    print("Kumulativer IR-Sprung (Rückgang) erkannt – prüfe auf Gesicht.")
                    screen.draw_text_center("Kumulativer IR-Sprung (Rückgang) erkannt – prüfe auf Gesicht.")
                    #break
                    time.sleep(0.5)
    
                    # Gesichtserkennung nach Objektentfernung
                    print("Prüfe auf Gesicht hinter dem Objekt...")
                    image = camera.capture()
                    camera.show_image(image)
                    face_coords = vision.find_face(image)

                    if face_coords is not None:
                        print("Gesicht erkannt!")
                        screen.draw_text_center("Gesicht erkannt!")
                        time.sleep(0.5)
                        global face_count
                        face_count += 1
                        screen.draw_text_center("Gesicht erkannt!")
                        log_aktion(logfile, aktion=str(face_count) + ". Gesicht erkannt! Zeile 583")
                        # Gesicht erkannt
                        (x, y, w, h) = face_coords
                        face_image = image[y:y+h, x:x+w]
                        
                        # Dateiname für Gesicht
                        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                        image_filename = "{}_face_{}.png".format(TEAM_NAME, timestamp)
                        image_path = os.path.join(SUBMISSION_DIR, image_filename)
                        
                        # Ordner sicherstellen
                        os.makedirs(SUBMISSION_DIR, exist_ok=True)
                        
                        # Bild speichern
                        cv2.imwrite(image_path, face_image)
                        print("📸 Gesicht gespeichert: {}".format(image_path))
                        

                    else:
                        screen.draw_text_center("KEIN Gesicht erkannt!")
                        print("Kein Gesicht erkannt.")

                    break

                time.sleep(0.5)

    finally:
        screen.happy()
        camera.close()
        print("Kamera geschlossen.")
        time.sleep(0.5)

    return True
 
    
def drive_course(zumi, total_time=60):
    global logfile
    global start_time
    if logfile is None:
        logfile = create_submission_file()

    print("Starte Parkour-Fahrt")
    screen.draw_text_center("Starte Parkour-Fahrt")
    start_time = time.time()
    log_aktion(logfile, aktion="Start der Fahrt Zeile 608")

    while time.time() - start_time < total_time:
        print("Starte neuen Block...")

        # Linien folgen für festgelegte Zeit
        follow_line(zumi)

In [ ]:
# Zeitbegrenzung noch auf alt 120 sec = 2 mins, -> neu = 6 mins -> 360
drive_course(zumi, total_time=360)

In [ ]:
zumi.stop()

In [ ]:
camera.close()

In [ ]:
upload_submission(logfile)  # Upload the latest submission
# Gesicht ins Git-Repo pushen
upload_face_image(image_path)